In [87]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from time import time

from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sksurv.metrics import concordance_index_censored

# Modelos
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

## Parametros

In [3]:
client_bq = bigquery.Client()

In [25]:
table = 'Diabetes_avicena_survival.diabetes_final_3_annos'

variables_with_outliers = ['edad','IMC','HDL','LDL','trigliceridos','perimetro_abdominal']

numeric_columns = ['edad','IMC','HDL','LDL','trigliceridos','perimetro_abdominal']
categoric_columns = ['genero_paciente','raza_paciente','nivel_academico_paciente','ant_cardio','med_hipertension','ant_familiar_dm','hace_ejercicio']
columns_not_in_count = ['ant_familiar_dm', 'raza_paciente','hace_ejercicio']
target = 'diabetes'

dict_var_categoricas = {
    # Nivel Academico
    "Ninguno" : 'ninguno',

    "Básica secundaria" : 'educacion_basica', 
    "Básica primaria" : 'educacion_basica',

    "Normalista" : 'educacion_media',
    "Bachillerato técnico" : 'educacion_media',
    "Técnica profesional" : 'educacion_media',
    "Tecnológica" : 'educacion_media',
    "Media académica o clásica" : 'educacion_media',

    "Profesional" : 'educacion_superior',
    "Especialización" : 'educacion_superior',
    "Preescolar" : 'educacion_superior',
    "Doctorado" : 'educacion_superior',
    "Maestría" : 'educacion_superior',
        
    # Ejercicio
    'Nunca' : 'No',
    '20 minutos' : '20 min',
    '40 minutos' : 'Mas de 20 min',
    '60 minutos' : 'Mas de 20 min',

    # Dicotomicas
    "1" : 'Si',
    '0' : 'No'
    

}

## Funciones

In [88]:
def escalar(data):
    
    # Crear el escalador
    scaler = MinMaxScaler()
    
    # Entrenar el escalador
    scaler.fit(data)
    
    # Re-escalar los datos
    df_escalado = pd.DataFrame(scaler.transform(data))
    
    return df_escalado

def outlier_label(value, limit):
    if value < limit[0]:
        return 'Abajo'
    elif value > limit[1]:
        return 'Arriba'
    else:
        return 'No'

def tag_outliers(data, variable):
    Q1 = data[variable].quantile(0.25)
    Q3 = data[variable].quantile(0.75)
    RIC = Q3 - Q1
    limit_inf = Q1 - (1.5 * RIC)
    limit_sup = Q3 + (1.5 * RIC)
    data[f'{variable}_outlier'] = data[variable].apply(lambda x: outlier_label(x, [limit_inf,limit_sup]))
    return data

def take_out_outliers(data, variables_with_outliers, verbose = True):
    
    data[variables_with_outliers] = data[variables_with_outliers].astype(float)
    for variable in variables_with_outliers:
        data_label = tag_outliers(data, variable)

    if verbose:
        for variable in variables_with_outliers:
            print(variable)
            conteos = data_label[f'{variable}_outlier'].value_counts().reset_index()
            total = conteos['count'].sum()
            conteos['Porcentaje'] = (conteos['count'] / total)*100
            display(conteos)

    columns_to_drop = [column + '_outlier' for column in variables_with_outliers]

    data_clean_outliers = data_label[(data_label.edad_outlier == 'No') &
                                     (data_label.IMC_outlier == 'No') &
                                     (data_label.HDL_outlier == 'No') &
                                     (data_label.LDL_outlier == 'No') &
                                     (data_label.trigliceridos_outlier == 'No') &
                                     (data_label.perimetro_abdominal_outlier == 'No')
                                    ]

    data_clean_outliers = data_clean_outliers.drop(columns = columns_to_drop)

    return data_clean_outliers

## Carga de datos

In [8]:
data = client_bq.query(f'SELECT * FROM {table}').result().to_dataframe()
print(f'Se trajo {data.shape} datos de pacientes')
data.head()

Se trajo (735003, 21) datos de pacientes


,numero_identificacion_paciente,year,month,fecha,edad,peso,talla,IMC,HDL,LDL,...,perimetro_abdominal,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio,diabetes,time_to_event
0,1003390652,2022,2,2022-02-01,26,105.233333,1.60,41.106770833,71.20,106.14,...,111.0,Femenino,Mestizo,Bachillerato técnico,0,0,0,None,0,34
1,1006578626,2022,2,2022-02-01,20,68.700000,1.77,21.928564589,56.25,143.00,...,NaN,Masculino,Otros,Profesional,1,0,0,None,0,36
2,1007218577,2022,2,2022-02-01,28,74.300000,1.78,23.450321929,45.10,96.64,...,NaN,Masculino,Otros,Ninguno,0,0,0,None,0,33
3,1010021506,2022,2,2022-02-01,36,71.000000,1.60,27.734375000,60.00,140.00,...,NaN,Masculino,Otros,Tecnológica,1,1,0,None,0,35
4,10241950,2022,2,2022-02-01,67,85.800000,1.72,29.002163332,41.00,82.00,...,105.0,Masculino,Otros,Básica secundaria,1,1,0,None,1,13


## Proceso

In [9]:
# Dataframe con los datos numericos
df_numerico = data[numeric_columns]
df_numerico['IMC'] = np.round(df_numerico['IMC'].astype(float),2)

df_numerico.head()

,edad,IMC,HDL,LDL,trigliceridos,perimetro_abdominal
0,26,41.11,71.20,106.14,117.80,111.0
1,20,21.93,56.25,143.00,100.05,NaN
2,28,23.45,45.10,96.64,64.68,NaN
3,36,27.73,60.00,140.00,117.00,NaN
4,67,29.00,41.00,82.00,203.00,105.0


In [27]:
# Dataframe con los datos categoricos
df_categorico = data[categoric_columns].astype(str)

df_categorico.hace_ejercicio = df_categorico.hace_ejercicio.replace(dict_var_categoricas)
df_categorico.nivel_academico_paciente = df_categorico.nivel_academico_paciente.replace(dict_var_categoricas)
df_categorico.ant_cardio = df_categorico.ant_cardio.replace(dict_var_categoricas)
df_categorico.med_hipertension = df_categorico.med_hipertension.replace(dict_var_categoricas)

# df_categorico = df_categorico.drop(columns = columns_not_in_count)

df_categorico

,genero_paciente,raza_paciente,nivel_academico_paciente,ant_cardio,med_hipertension,ant_familiar_dm,hace_ejercicio
0,Femenino,Mestizo,educacion_media,No,No,0,None
1,Masculino,Otros,educacion_superior,Si,No,0,None
2,Masculino,Otros,ninguno,No,No,0,None
3,Masculino,Otros,educacion_media,Si,Si,0,None
4,Masculino,Otros,educacion_basica,Si,Si,0,None
...,...,...,...,...,...,...,...
734998,Masculino,Otros,educacion_basica,No,No,0,None
734999,Masculino,Otros,educacion_superior,No,No,0,20 min
735000,Masculino,Otros,educacion_superior,No,Si,0,None
735001,Masculino,Otros,educacion_superior,Si,Si,0,None


In [28]:
# Conteo de las categorias de cada variable, con el re agrupamiento
for variable in categoric_columns + ['diabetes']:
    print(variable)
    if variable == 'diabetes':
        conteos = data[f'{variable}'].value_counts().reset_index()
    else:
        conteos = df_categorico[f'{variable}'].value_counts().reset_index()
    total = conteos['count'].sum()
    conteos['Porcentaje'] = (conteos['count'] / total)*100
    display(conteos)

genero_paciente


,genero_paciente,count,Porcentaje
0,Femenino,455992,62.039475
1,Masculino,279011,37.960525


raza_paciente


,raza_paciente,count,Porcentaje
0,Otros,682025,92.792138
1,Mestizo,33485,4.555764
2,Afrocolombiano,9233,1.256185
3,Raizales,3753,0.510610
4,Indígena,3371,0.458638
5,Palenquero,1826,0.248434
6,Rom/Gitano,1184,0.161088
7,None,126,0.017143


nivel_academico_paciente


...

,ant_familiar_dm,count,Porcentaje
0,0,733723,99.825851
1,1,1280,0.174149


hace_ejercicio


,hace_ejercicio,count,Porcentaje
0,None,700112,95.252945
1,No,19691,2.679037
2,20 min,7767,1.056730
3,Mas de 20 min,7433,1.011288


diabetes


,diabetes,count,Porcentaje
0,0,585512,79.661171
1,1,149491,20.338829


## Entrenamiento

In [39]:
# Diccionario con los cambios a las variables categoricas por numericas
dict_catergoricas = {
    'Femenino' : 0,
    'Masculino' : 1,
    "Mestizo" : 0,
    "Otros" : 1,
    "Afrocolombiano" : 2,
    "Raizales" : 3,
    "Indígena" : 4,
    "Palenquero" : 5,
    "Rom/Gitano" : 6,
    "Bachillerato técnico" : 0,
    "Básica secundaria" : 1,
    "Tecnológica" : 2,
    "Técnica profesional" : 3,
    "Profesional" : 4,
    "Ninguno" : 5,
    "Básica primaria" : 6,
    "Media académica o clásica" : 7,
    "Normalista" : 8,
    "Especialización" : 9,
    "Preescolar" : 10,
    "Doctorado" : 11,
    "Maestría" : 12,
    "20 minutos" : 0,
    "Nunca" : 1,
    "40 minutos" : 2,
    "60 minutos" : 3,
    'No' : 0,
    'Si' : 1,
    'educacion_media' : 2, 
    'educacion_superior' : 3, 
    'ninguno' : 0,
    'educacion_basica' : 1, 
    'None': 0 
}

In [45]:
# Cambiar datos nulos por el promedio de la columna
df_numerico_with_no_nan = df_numerico[::]
for column in df_numerico_with_no_nan.columns:
    df_numerico_with_no_nan[column] = df_numerico_with_no_nan[column].fillna(df_numerico_with_no_nan[column].mean())

# Escalar los valores
df_escalado = escalar(df_numerico_with_no_nan)
df_escalado.head()

,0,1,2,3,4,5
0,0.086022,0.041290,0.003158,0.006419,0.000305,0.411765
1,0.021505,0.021979,0.002495,0.007495,0.000259,0.330882
2,0.107527,0.023509,0.002000,0.006142,0.000168,0.330882
3,0.193548,0.027818,0.002661,0.007407,0.000303,0.330882
4,0.526882,0.029097,0.001819,0.005715,0.000526,0.384615


In [46]:
# Definir las variables categoricas con las que se va atrabajar
df_categorico_no_columns = df_categorico.drop(columns = columns_not_in_count)
df_categorico_no_columns

,genero_paciente,nivel_academico_paciente,ant_cardio,med_hipertension
0,Femenino,educacion_media,No,No
1,Masculino,educacion_superior,Si,No
2,Masculino,ninguno,No,No
3,Masculino,educacion_media,Si,Si
4,Masculino,educacion_basica,Si,Si
...,...,...,...,...
734998,Masculino,educacion_basica,No,No
734999,Masculino,educacion_superior,No,No
735000,Masculino,educacion_superior,No,Si
735001,Masculino,educacion_superior,Si,Si


In [ ]:
# unir el df escalado y las variables categoricas
data_to_train = df_escalado.reset_index().merge(df_categorico_no_columns.reset_index(), on = 'index', how = 'left')
data_to_train.drop(columns = ['index'], inplace=True)

# Convertir el nombre de las columnas a string
data_to_train.columns = data_to_train.columns.astype(str)

# Cambiar variables categoricas a numericas
data_to_train.genero_paciente = data_to_train.genero_paciente.replace(dict_catergoricas)
data_to_train.nivel_academico_paciente = data_to_train.nivel_academico_paciente.replace(dict_catergoricas)
data_to_train.ant_cardio = data_to_train.ant_cardio.replace(dict_catergoricas)
data_to_train.med_hipertension = data_to_train.med_hipertension.replace(dict_catergoricas)
# data_to_train.hace_ejercicio = data_to_train.hace_ejercicio.replace(dict_catergoricas)
# data_to_train.raza_paciente = data_to_train.raza_paciente.replace(dict_catergoricas)

# Mostrar datos a usar en el entrenamiento
data_to_train

In [51]:
y = data[['diabetes','time_to_event']]
y['target'] = y.apply(lambda x: (bool(x.diabetes), x.time_to_event), axis = 1)
y = y['target']
y = np.array(y, dtype=[('event', np.bool_), ('time', np.int32)])

display(y[:5])

X = data_to_train

display(X.head(5))

random_state = 20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)

print(f'Las dimensiones del entrenamiento son {X_train.shape} para X_train, {y_train.shape} para y_train')
print(f'Las dimensiones del entrenamiento son {X_test.shape} para X_test, {y_test.shape} para y_test')

# rsf = RandomSurvivalForest(
#     n_estimators=1000, min_samples_split=10, min_samples_leaf=15, n_jobs=-1, random_state=random_state
# )

rsf = RandomSurvivalForest(
    max_depth=100, min_samples_leaf=50, min_samples_split=30,
                     n_estimators=50, n_jobs=-1
)

rsf.fit(X_train, y_train)

print('El modelo tiene un valor score de :',rsf.score(X_test, y_test))

surv = rsf.predict_survival_function(X_test)
tiempos_supervivencia = [int(y_test[i][1]) for i in range(len(y_test))]
eventos = [bool(y_test[i][0]) for i in range(len(y_test))]
tiempo_mediano = np.median(tiempos_supervivencia)
puntuaciones_riesgo = -np.log([f(tiempo_mediano) for f in surv])

c_index = concordance_index_censored(eventos, tiempos_supervivencia, puntuaciones_riesgo)

c_index[0]

array([(False, 34), (False, 36), (False, 33), (False, 35), ( True, 13)],
      dtype=[('event', '?'), ('time', '<i4')])

,0,1,2,3,4,5,genero_paciente,nivel_academico_paciente,ant_cardio,med_hipertension
0,0.086022,0.041290,0.003158,0.006419,0.000305,0.411765,0,2,0,0
1,0.021505,0.021979,0.002495,0.007495,0.000259,0.330882,1,3,1,0
2,0.107527,0.023509,0.002000,0.006142,0.000168,0.330882,1,0,0,0
3,0.193548,0.027818,0.002661,0.007407,0.000303,0.330882,1,2,1,1
4,0.526882,0.029097,0.001819,0.005715,0.000526,0.384615,1,1,1,1


Las dimensiones del entrenamiento son (551252, 10) para X_train, (551252,) para y_train
Las dimensiones del entrenamiento son (183751, 10) para X_test, (183751,) para y_test
El modelo tiene un valor score de : 0.5994707686844261


0.5998579113526519

## Entrenamiento sin outliers

In [89]:
df_no_outliers = take_out_outliers(data, variables_with_outliers, verbose = False)

In [90]:
df_no_outliers.shape

(657034, 21)

In [109]:
# Dataframe con los datos numericos
df_numerico = df_no_outliers[numeric_columns]
df_numerico['IMC'] = np.round(df_numerico['IMC'].astype(float),2)

# print(df_numerico.shape)
# display(df_numerico.head())

# Dataframe con los datos categoricos
df_categorico = df_no_outliers[categoric_columns].astype(str)

df_categorico.hace_ejercicio = df_categorico.hace_ejercicio.replace(dict_var_categoricas)
df_categorico.nivel_academico_paciente = df_categorico.nivel_academico_paciente.replace(dict_var_categoricas)
df_categorico.ant_cardio = df_categorico.ant_cardio.replace(dict_var_categoricas)
df_categorico.med_hipertension = df_categorico.med_hipertension.replace(dict_var_categoricas)

# df_categorico = df_categorico.drop(columns = columns_not_in_count)

# print(df_categorico.shape)
# display(df_categorico.head())

# Cambiar datos nulos por el promedio de la columna
df_numerico_with_no_nan = df_numerico[::]
for column in df_numerico_with_no_nan.columns:
    df_numerico_with_no_nan[column] = df_numerico_with_no_nan[column].fillna(df_numerico_with_no_nan[column].mean())

# Escalar los valores
df_escalado = escalar(df_numerico_with_no_nan)
df_escalado.head()

# Definir las variables categoricas con las que se va atrabajar
df_categorico_no_columns = df_categorico.drop(columns = columns_not_in_count).reset_index()
df_categorico_no_columns.drop(columns = 'index', inplace = True)
df_categorico_no_columns

# unir el df escalado y las variables categoricas
data_to_train = df_escalado.reset_index().merge(df_categorico_no_columns.reset_index(), on = 'index', how = 'left')
data_to_train.drop(columns = ['index'], inplace=True)

# Convertir el nombre de las columnas a string
data_to_train.columns = data_to_train.columns.astype(str)

# Cambiar variables categoricas a numericas
data_to_train.genero_paciente = data_to_train.genero_paciente.replace(dict_catergoricas)
data_to_train.nivel_academico_paciente = data_to_train.nivel_academico_paciente.replace(dict_catergoricas)
data_to_train.ant_cardio = data_to_train.ant_cardio.replace(dict_catergoricas)
data_to_train.med_hipertension = data_to_train.med_hipertension.replace(dict_catergoricas)
# data_to_train.hace_ejercicio = data_to_train.hace_ejercicio.replace(dict_catergoricas)
# data_to_train.raza_paciente = data_to_train.raza_paciente.replace(dict_catergoricas)

# Mostrar datos a usar en el entrenamiento
data_to_train

,0,1,2,3,4,5,genero_paciente,nivel_academico_paciente,ant_cardio,med_hipertension
0,0.024096,0.269439,0.644531,0.629771,0.312852,0.492912,1,3,1,0
1,0.120482,0.332640,0.470313,0.408588,0.202251,0.492912,1,0,0,0
2,0.216867,0.510603,0.703125,0.615458,0.365854,0.492912,1,2,1,1
3,0.590361,0.563410,0.406250,0.338740,0.634772,0.700405,1,1,1,1
4,0.578313,0.429522,0.492188,0.168416,0.340838,0.492912,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
657029,0.409639,0.769647,0.234375,0.500954,0.859912,0.700405,1,1,0,0
657030,0.698795,0.299792,0.514062,0.260592,0.288931,0.165992,1,3,0,0
657031,0.626506,0.307277,0.421875,0.224237,0.572233,0.492912,1,3,0,1
657032,0.771084,0.563825,0.375000,0.682252,0.553471,0.492912,1,3,1,1


In [110]:
# Conteo de las categorias de cada variable, con el re agrupamiento
for variable in categoric_columns + ['diabetes']:
    print(variable)
    if variable == 'diabetes':
        conteos = df_no_outliers[f'{variable}'].value_counts().reset_index()
    else:
        conteos = df_categorico[f'{variable}'].value_counts().reset_index()
    total = conteos['count'].sum()
    conteos['Porcentaje'] = (conteos['count'] / total)*100
    display(conteos)

genero_paciente


,genero_paciente,count,Porcentaje
0,Femenino,408440,62.164211
1,Masculino,248594,37.835789


raza_paciente


,raza_paciente,count,Porcentaje
0,Otros,609344,92.741624
1,Mestizo,30256,4.604937
2,Afrocolombiano,8219,1.250925
3,Raizales,3361,0.511541
4,Indígena,3025,0.460402
5,Palenquero,1645,0.250368
6,Rom/Gitano,1069,0.162701
7,None,115,0.017503


nivel_academico_paciente


...

,ant_familiar_dm,count,Porcentaje
0,0,655900,99.827406
1,1,1134,0.172594


hace_ejercicio


,hace_ejercicio,count,Porcentaje
0,None,626369,95.332814
1,No,17150,2.610215
2,20 min,6939,1.056110
3,Mas de 20 min,6576,1.000861


diabetes


,diabetes,count,Porcentaje
0,0,525603,79.996317
1,1,131431,20.003683


In [106]:
t_1 = time()

y = df_no_outliers[['diabetes','time_to_event']]
y['target'] = y.apply(lambda x: (bool(x.diabetes), x.time_to_event), axis = 1)
y = y['target']
y = np.array(y, dtype=[('event', np.bool_), ('time', np.int32)])

display(y[:5])

X = data_to_train

display(X.head(5))

random_state = 20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state)

print(f'Las dimensiones del entrenamiento son {X_train.shape} para X_train, {y_train.shape} para y_train')
print(f'Las dimensiones del entrenamiento son {X_test.shape} para X_test, {y_test.shape} para y_test')

# rsf = RandomSurvivalForest(
#     n_estimators=1000, min_samples_split=10, min_samples_leaf=15, n_jobs=-1, random_state=random_state
# )

rsf = RandomSurvivalForest(
    max_depth=100, min_samples_leaf=50, min_samples_split=30,
                     n_estimators=50, n_jobs=-1
)

rsf.fit(X_train, y_train)

print('El modelo tiene un valor score de :',rsf.score(X_test, y_test))

surv = rsf.predict_survival_function(X_test)
tiempos_supervivencia = [int(y_test[i][1]) for i in range(len(y_test))]
eventos = [bool(y_test[i][0]) for i in range(len(y_test))]
tiempo_mediano = np.median(tiempos_supervivencia)
puntuaciones_riesgo = -np.log([f(tiempo_mediano) for f in surv])

c_index = concordance_index_censored(eventos, tiempos_supervivencia, puntuaciones_riesgo)

print(c_index[0])

print(f'Se demoro un total de {((time() - t_1)/60)} minutos')

array([(False, 36), (False, 33), (False, 35), ( True, 13), ( True,  1)],
      dtype=[('event', '?'), ('time', '<i4')])

,0,1,2,3,4,5,genero_paciente,nivel_academico_paciente,ant_cardio,med_hipertension
0,0.024096,0.269439,0.644531,0.629771,0.312852,0.492912,1,3,1,0
1,0.120482,0.332640,0.470313,0.408588,0.202251,0.492912,1,0,0,0
2,0.216867,0.510603,0.703125,0.615458,0.365854,0.492912,1,2,1,1
3,0.590361,0.563410,0.406250,0.338740,0.634772,0.700405,1,1,1,1
4,0.578313,0.429522,0.492188,0.168416,0.340838,0.492912,1,1,1,1


Las dimensiones del entrenamiento son (492775, 10) para X_train, (492775,) para y_train
Las dimensiones del entrenamiento son (164259, 10) para X_test, (164259,) para y_test
El modelo tiene un valor score de : 0.5956498248845631
0.5958747908321835
Se demoro un total de 49.77359567483266 minutos
